# G06 - Wall Jump
---

## Index
[1.Introduction](#intro) <br>
[2.Case analysis](#oldCase) <br>
[3.Performance analysis](#performance) <br>
[4.New case proposal](#newCase) <br>
[5.Team](#team)

## <a name="intro"></a>1. Introduction
This notebook has all the information related to <b>G06</b>'s delivery 1. Here you can find an analysis of the <b>Wall Jump</b> example of Unity's ml-agents, as well as a new case proposal of the same example, with all information needed to reproduce it by yourself.

This notebook also works as a post mortem for our new proposal's implementation.

|![AlexRivero](g06-img/Alex.png)|![AlexRivero](g06-img/Alex.png)|
|---|---|
|Alex Rivero Ferràs|David Recuero Redrado|
|alexriveroferras@enti.cat|davidrecueroredrado@enti.cat|

## <a name="oldCase"></a>2. Case analysis

### Rewards
In this example there are 3 rewards given to the agent depending on its performance solving the problem:
- <span style="color:green">+1.0, if the agent reaches the goal</span> - Given to the agent in the <i>OnTriggerStay()</i> (using the collider of the goal)
- <span style="color:red">-0.0005, for every step the agent does</span> - Given to the agent in <i>MoveAgent()</i>. This reward motivates the agent to find the optimal path to the goal (using as less movements as possible)
- <span style="color:red">-1.0, if the agent or the cube falls off the platform</span> - Given to the agent in <i>AgentAction()</i>
![MoveAgent](g06-img/Rewards.png)

### Actions
The agent can take 4 different main actions, each of them with multiple options:
<img style="float: right;" src="g06-img/Movement.gif">
- Forward movement
    - Forward
    - Backwards 
    - No action
- Side movement
    - Left
    - Right
    - No action
- Rotation
    - Left
    - Right
    - No action
- Jump
    - Jump
    - No action
    
### Observations
The agent has 2 different observations in the <i>CollectObservations()</i> function: the position of the agent and a boolean indicating if the agent is grounded (as the agent only receives the reward if it stays on the goal being grounded). It also has 14 ray casts each detecting 4 possible objects.

### How it all works
In this example the agent has to reach a green area tagged as "goal". The environment has 3 different states:
- The goal accessible for the agent <b>without any wall</b> blocking it.
- A <b>small-sized wall</b> blocking the way to the goal. In this case the agent can jump over the wall with a simple jump.
- A <b>big-sized wall</b> blocking the way to the goal. In this case the agent has to push a cube against the wall, jump on the cube and then jump over the wall.

As this example has 3 different states it would take too long to train it using the "hard way" (trying randomly every situation). That's why it uses curriculum learning.<br>
Curriculum learning uses progression to train the agent. In this example, the wall scales up when the agent reaches a threshold. The agent learns to solve the 3 states progressively, making it easier and faster to train.

To do so, the agent uses 2 brains and, depending on the current case, the brain is passed to the behavior parameters using the <i>ConfigureAgent()</i> function.

```c#
void ConfigureAgent(int config)
    {
        var localScale = wall.transform.localScale;
        if (config == 0)
        {
            localScale = new Vector3(
                localScale.x,
                Academy.Instance.FloatProperties.GetPropertyWithDefault("no_wall_height", 0),
                localScale.z);
            wall.transform.localScale = localScale;
            GiveModel("SmallWallJump", noWallBrain);
        }
        else if (config == 1)
        {
            localScale = new Vector3(
                localScale.x,
                Academy.Instance.FloatProperties.GetPropertyWithDefault("small_wall_height", 4),
                localScale.z);
            wall.transform.localScale = localScale;
            GiveModel("SmallWallJump", smallWallBrain);
        }
        else
        {
            var min = Academy.Instance.FloatProperties.GetPropertyWithDefault("big_wall_min_height", 8);
            var max = Academy.Instance.FloatProperties.GetPropertyWithDefault("big_wall_max_height", 8);
            var height = min + Random.value * (max - min);
            localScale = new Vector3(
                localScale.x,
                height,
                localScale.z);
            wall.transform.localScale = localScale;
            GiveModel("BigWallJump", bigWallBrain);
        }
    }
```

The <i>SmallWallJump</i> brain is used both for when there's no wall and when the wall is low enough to jump over it. The <i>BigWallJump</i> brain is used when the wall is too high and the agent needs the cube to jump over it.

Here's a link to the repository's docs where curriculum learning is explained: https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Training-Curriculum-Learning.md

## <a name="performance"></a>3. Performance analysis

## <a name="newCase"></a>4. New case proposal

## <a name="team"></a>5. Team

|![AlexRivero](g06-img/Alex.png)|![AlexRivero](g06-img/Alex.png)|
|---|---|
|Alex Rivero Ferràs|David Recuero Redrado|
|alexriveroferras@enti.cat|davidrecueroredrado@enti.cat|